In [1]:
from keras.layers import Input, Lambda, Dense, Flatten, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import optimizers
from keras.regularizers import l2

import numpy as np
from glob import glob
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Define your image size
IMAGE_SIZE = [224, 224]


In [3]:
# Load the pre-trained VGG16 model with custom head
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58889256/58889256 [==============================] - 0s 0us/step


In [4]:
# Create the model
x = Flatten()(vgg.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
x = Dropout(0.5)(x)  # Increase dropout
prediction = Dense(58, activation='softmax')(x)  # Make sure the output matches your number of classes
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
# Use a lower learning rate
adam = optimizers.Adam(learning_rate=0.0001)

model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=['accuracy'])

In [6]:
# Increase data augmentation
train_generator = ImageDataGenerator(
    validation_split=0.20,
    rescale=1./255,
    shear_range=0.2,  # Increase shear range
    zoom_range=0.2,  # Increase zoom range
    horizontal_flip=True,  # Add horizontal flipping
    preprocessing_function=preprocess_input
)
image_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

# Create Data Loaders
train_loader = train_generator.flow_from_directory(
    directory="/kaggle/input/training-handwritten-marathi-character-fusion/Data_set",
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    class_mode='categorical',
    subset='training',
    seed=42
)

valid_loader = train_generator.flow_from_directory(
    directory="/kaggle/input/training-handwritten-marathi-character-fusion/Data_set",
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    class_mode='categorical',
    subset='validation',
    seed=42
)

test_loader = image_generator.flow_from_directory(
    directory="/kaggle/input/handwritten-marathi-character-dataset-of-4-people/Testing_Data",
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=42
)


Found 13348 images belonging to 58 classes.
Found 3305 images belonging to 58 classes.
Found 232 images belonging to 58 classes.


In [7]:
# Implement early stopping
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

# Train and Test Steps
steps_train = len(train_loader)
steps_valid = len(valid_loader)

In [8]:
model_history = model.fit_generator(
    train_loader,
    validation_data=valid_loader,
    epochs=100,
    steps_per_epoch=steps_train,
    validation_steps=steps_valid,
    callbacks=[early_stopping]  # Use early stopping
)



/tmp/ipykernel_28/875508701.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_history = model.fit_generator(


Epoch 1/100
418/418 [==============================] - 372s 843ms/step - loss: 4.0161 - accuracy: 0.0727 - val_loss: 2.8660 - val_accuracy: 0.3032
Epoch 2/100
418/418 [==============================] - 299s 714ms/step - loss: 2.7141 - accuracy: 0.3121 - val_loss: 1.3730 - val_accuracy: 0.6672
Epoch 3/100
418/418 [==============================] - 298s 712ms/step - loss: 1.6193 - accuracy: 0.5677 - val_loss: 0.8767 - val_accuracy: 0.7761
Epoch 4/100
418/418 [==============================] - 297s 709ms/step - loss: 1.0589 - accuracy: 0.7118 - val_loss: 0.7060 - val_accuracy: 0.8218
Epoch 5/100
418/418 [==============================] - 298s 714ms/step - loss: 0.7829 - accuracy: 0.7961 - val_loss: 0.6216 - val_accuracy: 0.8393
Epoch 6/100
418/418 [==============================] - 294s 703ms/step - loss: 0.6385 - accuracy: 0.8364 - val_loss: 0.5906 - val_accuracy: 0.8575
Epoch 7/100
418/418 [==============================] - 295s 705ms/step - loss: 0.5409 - accuracy: 0.8679 - val_loss: 0

In [19]:
results = model.evaluate(test_loader)
print("Test Accuracy:", results[1])

Test Accuracy: 0.7025862336158752
